<a href="https://colab.research.google.com/github/IlyasHidayatR/Python_SimpleProgram/blob/main/ChatbotAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Data

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import json

with open("intents.json") as file:
    data = json.load(file)

words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

output_empty = [0] * len(labels)

for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = list(output_empty)
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

with open("data.json", "w") as file:
    json.dump({"training": training.tolist(), "output": output.tolist()}, file, indent=4)

# Define the TensorFlow model
input_size = len(training[0])
output_size = len(output[0])

X = tf.placeholder(tf.float32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, output_size])

hidden_size = 8

weights = {
    'hidden': tf.Variable(tf.random_normal([input_size, hidden_size])),
    'output': tf.Variable(tf.random_normal([hidden_size, output_size]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([hidden_size])),
    'output': tf.Variable(tf.random_normal([output_size]))
}

hidden_layer = tf.add(tf.matmul(X, weights['hidden']), biases['hidden'])
hidden_layer = tf.nn.relu(hidden_layer)

output_layer = tf.matmul(hidden_layer, weights['output']) + biases['output']

# Define the loss and optimizer
# Define the loss and optimizer
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=Y))
    optimizer = tf.train.AdamOptimizer(name='Adam-Optimizer').minimize(cost)

# Initialize variables and create a session
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training loop
    epochs = 1000
    batch_size = 8
    for epoch in range(epochs):
        for i in range(0, len(training), batch_size):
            batch_x = training[i:i + batch_size]
            batch_y = output[i:i + batch_size]
            sess.run(optimizer, feed_dict={X: batch_x, Y: batch_y})

        if (epoch + 1) % 100 == 0:
            c = sess.run(cost, feed_dict={X: training, Y: output})
            print(f"Epoch {epoch+1}/{epochs}, cost: {c:.4f}")

    print("Training completed!")

    # Save the model
    saver = tf.train.Saver()
    saver.save(sess, "model1.tf")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Epoch 100/1000, cost: 1.0454
Epoch 200/1000, cost: 0.4566
Epoch 300/1000, cost: 0.2358
Epoch 400/1000, cost: 0.1041
Epoch 500/1000, cost: 0.0482
Epoch 600/1000, cost: 0.0211
Epoch 700/1000, cost: 0.0101
Epoch 800/1000, cost: 0.0038
Epoch 900/1000, cost: 0.0011
Epoch 1000/1000, cost: 0.0006
Training completed!


# Fungsi menguji chatbot

In [ ]:
def chat_with_bot(sess, words, labels, input_text):
    input_text = input_text.lower()
    input_words = nltk.word_tokenize(input_text)
    input_words = [stemmer.stem(word) for word in input_words]

    input_bag = [0] * len(words)
    for word in input_words:
        if word in words:
            input_bag[words.index(word)] = 1

    input_bag = np.array(input_bag)
    input_bag = input_bag.reshape(1, -1)  # Reshape to a 2D array

    results = sess.run(output_layer, feed_dict={X: input_bag})
    results_index = np.argmax(results)
    tag = labels[results_index]

    for intent in data["intents"]:
        if intent['tag'] == tag:
            responses = intent['responses']

    return random.choice(responses)

In [ ]:
import random
# Load the saved model
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('model1.tf.meta')
    saver.restore(sess, "model1.tf")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break
        response = chat_with_bot(sess, words, labels, user_input)
        print("Bot:", response)

You: Hai
Bot: Halo! Terima kasih, semoga juga untukmu.
You: quit


# Fungsi uji chatbot dalam bentuk API

In [ ]:
from flask import Flask, request, jsonify
import random

# Import semua yang diperlukan, termasuk model dan fungsi chat_with_bot

app = Flask(__name__)

# Load the saved model
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('model1.tf.meta')
    saver.restore(sess, "model1.tf")

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('user_input')
    if user_input.lower() == "quit":
        response = "Bot: Goodbye!"
    else:
        response = chat_with_bot(sess, words, labels, user_input)
    return jsonify({'response': response})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
